# Activation foil analysis

In [1]:
from libra_toolbox.neutron_detection.activation_foils import *
from data import foil_data
import pandas as pd
import pint_pandas

In [2]:
# make a pandas dataframe based on the foil data with columns for run, generator, flux and error

irradiations = [
    {"t_on": 0, "t_off": 12 * settings.ureg.h},
    {"t_on": 24 * settings.ureg.h, "t_off": 36 * settings.ureg.h},
]

data = {
    "run": [],
    "generator": [],
    "flux": [],
    "error": [],
}

for name, foil in foil_data.items():
    flux = explicit.get_neutron_flux(foil, irradiations)
    error = explicit.get_neutron_flux_error(foil)
    data["run"].append(foil["run"])
    data["generator"].append(foil["generator"])
    data["flux"].append(flux.to(settings.ureg.s**-1))
    data["error"].append(error)
data["flux"] = pd.Series(data["flux"], dtype="pint[s**-1]")
pd.options.display.float_format = '{:,.2e}'.format
df = pd.DataFrame(data)
df.pint.dequantify()

,run,generator,flux,error
unit,No Unit,No Unit,1 / second,No Unit
0,4,P383,2.61e+08,8.81e-02
1,4,P383,2.67e+08,8.81e-02
2,4,A325,9.84e+07,8.82e-02
3,4,A325,1.06e+08,8.82e-02
4,5,A325,1.15e+08,8.82e-02
5,5,A325,1.17e+08,8.82e-02
6,5,P383,2.68e+08,8.81e-02
7,5,P383,2.74e+08,8.81e-02
8,6,A325,1.06e+08,8.82e-02


In [3]:
#for each run number compute the average per generator
grouped_multiple = df.groupby(['run', 'generator']).agg({'flux': ['mean', 'min', 'max']})
grouped_multiple.columns = ['flux_mean', 'flux_min', 'flux_max']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple.pint.dequantify()


,run,generator,flux_mean,flux_min,flux_max
unit,No Unit,No Unit,1 / second,1 / second,1 / second
0,4,A325,1.02e+08,9.84e+07,1.06e+08
1,4,P383,2.64e+08,2.61e+08,2.67e+08
2,5,A325,1.16e+08,1.15e+08,1.17e+08
3,5,P383,2.71e+08,2.68e+08,2.74e+08
4,6,A325,1.06e+08,1.06e+08,1.07e+08
5,6,P383,2.48e+08,2.46e+08,2.49e+08
